In [1]:
import pandas as pd
from statsmodels.sandbox.stats.multicomp import multipletests
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as po
import plotly.io as pio
import numpy as np
import h5py

In [2]:
# Version 1
mult_reg_results_n_v, _, _,rsquared_n_v  = pd.read_pickle('/auto/data2/oelmas/RSA_Results/Naive/Video/correlation_hamming_regression_results.pkl')
#mult_reg_results_n_mf, _, _,rsquared_n_mf  = pd.read_pickle('/auto/data2/oelmas/RSA_Results/Naive/Still_MF/correlation_hamming_regression_results.pkl')
mult_reg_results_n_ff, _, _,rsquared_n_ff  = pd.read_pickle('/auto/data2/oelmas/RSA_Results/Naive/Still_FF/correlation_hamming_regression_results.pkl')

mult_reg_results_p_v, _, _,rsquared_p_s  = pd.read_pickle('/auto/data2/oelmas/RSA_Results/Prior/Video/correlation_hamming_regression_results.pkl')
mult_reg_results_p_s, _, _,rsquared_p_v  = pd.read_pickle('/auto/data2/oelmas/RSA_Results/Prior/Still/correlation_hamming_regression_results.pkl')

mult_reg_results = [mult_reg_results_n_v, mult_reg_results_n_ff,mult_reg_results_p_v, mult_reg_results_p_s]
mult_reg_results = pd.concat(mult_reg_results)

In [3]:
# Plots significant regressor beta values 
# If used need to change
def visualize_significant_regressors(mult_reg_results, electrode_region, exp_type, stm_type):
    er_idx = mult_reg_results.electrode_region==electrode_region
    ex_idx = mult_reg_results.experiment_type==exp_type
    st_idx = mult_reg_results.stimuli_type==stm_type
    reg_df = mult_reg_results[ex_idx & st_idx & er_idx]
    
    if reg_df.empty:
        print("Empty df: " +exp_type+"_"+stm_type+"_"+electrode_region)
    else:
        sig_idx = multipletests(reg_df["p_value"], alpha=0.05, method='bonferroni')[0]
        reg_df.loc[:, 'significant'] = sig_idx.tolist()
        
        
        sig_reg_df = reg_df.loc[reg_df['significant']]
        sig_reg_df =sig_reg_df[sig_reg_df.beta_value>0]
        if not sig_reg_df.empty:
            fig = px.line(sig_reg_df, 
                         x="time", y="beta_value",  color="model_name",
                        title=exp_type+" "+stm_type+" "+electrode_region+" timepoints where the regressors reached significance")

            fig.update_yaxes(title_text="Regressors")
            fig.update_xaxes(range=[-200, 600], title_text="Time ms (relative to stimulus onset)")
            
            # Change in server
            fig.write_image(exp_type+"_"+stm_type+"_"+electrode_region+"_significant_beta_vals.png")
            return sig_reg_df

In [4]:
mult_reg_results = mult_reg_results[mult_reg_results.model_name != 'constant']

for electrode_region in ["occipital",'parietal']:
    for exp_type in mult_reg_results.experiment_type.unique():
        for stm_type in mult_reg_results.stimuli_type.unique():
            if stm_type == "still-mf":
                continue
            visualize_significant_regressors(mult_reg_results, electrode_region, exp_type, stm_type)

/auto/k2/oelmas/anaconda/envs/agent_perception_rsa/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/auto/k2/oelmas/anaconda/envs/agent_perception_rsa/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
v2_mult_reg_results = v2_mult_reg_results[v2_mult_reg_results.model_name != 'constant']
for electrode_region in v2_mult_reg_results.electrode_region.unique():
    for exp_type in v2_mult_reg_results.experiment_type.unique():
        for stm_type in v2_mult_reg_results.stimuli_type.unique():
            if exp_type == "prior" and stm_type == "still-mf":
                continue
            visualize_significant_regressors(v2_mult_reg_results, electrode_region, exp_type, stm_type, 2)

NameError: name 'v2_mult_reg_results' is not defined

RSA RESULTS